# 🏆 SOCAR Hackathon 2025 - Handwriting OCR Training

**Complete implementation of Hybrid OCR Architecture on real handwriting data**

This notebook:
1. Downloads real handwriting dataset from Kaggle
2. Implements preprocessing pipeline
3. Fine-tunes TrOCR model
4. Implements ensemble approach
5. Evaluates performance
6. Visualizes results

**Dataset**: Handwritten2Text Training Dataset (Kaggle)

---

## 📋 Table of Contents
1. [Setup & Installation](#setup)
2. [Dataset Download](#dataset)
3. [Data Exploration](#exploration)
4. [Preprocessing Pipeline](#preprocessing)
5. [Model Training](#training)
6. [Ensemble Implementation](#ensemble)
7. [Evaluation](#evaluation)
8. [Demo & Visualization](#demo)

---

## 1. Setup & Installation 🔧

Install all required packages for our hybrid OCR system.

In [ ]:
# Check if running in Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✅ Running in Google Colab")
    # Check GPU
    !nvidia-smi -L
else:
    print("⚠️ Not in Colab - some features may not work")

In [ ]:
%%capture
# Install required packages (suppress output)
!pip install -q kagglehub
!pip install -q transformers>=4.35.0
!pip install -q datasets
!pip install -q torch torchvision
!pip install -q Pillow
!pip install -q opencv-python
!pip install -q albumentations
!pip install -q matplotlib seaborn
!pip install -q scikit-learn
!pip install -q jiwer  # For CER/WER calculation
!pip install -q tqdm
!pip install -q accelerate

print("✅ All packages installed!")

In [ ]:
# Import libraries
import os
import json
import random
from pathlib import Path
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import (
    VisionEncoderDecoderModel,
    TrOCRProcessor,
    AutoTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator
)

from tqdm.auto import tqdm
from jiwer import cer, wer

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")

---

## 2. Dataset Download 📥

Download the handwriting dataset from Kaggle using kagglehub.

In [ ]:
import kagglehub

# Download latest version
print("📥 Downloading dataset from Kaggle...")
print("This may take a few minutes...\n")

path = kagglehub.dataset_download("chaimaourgani/handwritten2text-training-dataset")

print(f"\n✅ Dataset downloaded!")
print(f"📁 Path to dataset files: {path}")

# List files
dataset_path = Path(path)
all_files = list(dataset_path.rglob('*'))
print(f"\n📊 Total files: {len(all_files)}")
print("\n📂 Directory structure:")
for item in list(dataset_path.glob('*'))[:10]:
    print(f"  - {item.name}")

---

## 3. Data Exploration 🔍

Let's explore the dataset structure and contents.

In [ ]:
# Find image and text files
image_files = list(dataset_path.rglob('*.jpg')) + list(dataset_path.rglob('*.png'))
text_files = list(dataset_path.rglob('*.txt'))

print(f"📷 Image files: {len(image_files)}")
print(f"📝 Text files: {len(text_files)}")

# Show some examples
if image_files:
    print(f"\n📷 Sample image files:")
    for img in image_files[:5]:
        print(f"  - {img.name}")

if text_files:
    print(f"\n📝 Sample text files:")
    for txt in text_files[:5]:
        print(f"  - {txt.name}")

In [ ]:
# Create dataset mapping
def find_matching_pairs(dataset_path):
    """Find matching image-text pairs."""
    pairs = []
    
    # Look for common patterns
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    
    for img_path in dataset_path.rglob('*'):
        if img_path.suffix.lower() in image_extensions:
            # Try to find corresponding text file
            txt_path = img_path.with_suffix('.txt')
            if txt_path.exists():
                pairs.append({
                    'image': str(img_path),
                    'text_file': str(txt_path)
                })
    
    return pairs

# Find pairs
data_pairs = find_matching_pairs(dataset_path)
print(f"✅ Found {len(data_pairs)} image-text pairs")

# If no pairs found, try alternative method
if len(data_pairs) == 0:
    print("\n⚠️ No direct pairs found, trying alternative structure...")
    
    # Check for CSV or JSON metadata
    csv_files = list(dataset_path.rglob('*.csv'))
    json_files = list(dataset_path.rglob('*.json'))
    
    if csv_files:
        print(f"\n📊 Found CSV files: {[f.name for f in csv_files]}")
        # Try to load first CSV
        df = pd.read_csv(csv_files[0])
        print(f"\nCSV columns: {df.columns.tolist()}")
        print(f"\nFirst few rows:")
        print(df.head())
    
    if json_files:
        print(f"\n📄 Found JSON files: {[f.name for f in json_files]}")

In [ ]:
# Visualize some samples
def visualize_samples(data_pairs, n_samples=6):
    """Visualize random samples from dataset."""
    if len(data_pairs) == 0:
        print("⚠️ No data pairs to visualize")
        return
    
    samples = random.sample(data_pairs, min(n_samples, len(data_pairs)))
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, pair in enumerate(samples):
        if idx >= 6:
            break
            
        # Load image
        img = Image.open(pair['image'])
        
        # Load text
        with open(pair['text_file'], 'r', encoding='utf-8') as f:
            text = f.read().strip()
        
        # Display
        axes[idx].imshow(img, cmap='gray')
        axes[idx].axis('off')
        axes[idx].set_title(f"Text: {text[:50]}...", fontsize=10)
    
    plt.tight_layout()
    plt.suptitle('Sample Handwritten Images with Ground Truth', 
                 fontsize=14, y=1.02)
    plt.show()

# Visualize
visualize_samples(data_pairs)

In [ ]:
# Dataset statistics
def analyze_dataset(data_pairs):
    """Analyze dataset characteristics."""
    if len(data_pairs) == 0:
        print("⚠️ No data to analyze")
        return
    
    text_lengths = []
    image_sizes = []
    
    print("📊 Analyzing dataset...")
    for pair in tqdm(data_pairs[:1000]):  # Analyze subset for speed
        # Text length
        with open(pair['text_file'], 'r', encoding='utf-8') as f:
            text = f.read().strip()
            text_lengths.append(len(text))
        
        # Image size
        img = Image.open(pair['image'])
        image_sizes.append(img.size)
    
    # Statistics
    print(f"\n📏 Text Length Statistics:")
    print(f"  Min: {min(text_lengths)} characters")
    print(f"  Max: {max(text_lengths)} characters")
    print(f"  Mean: {np.mean(text_lengths):.1f} characters")
    print(f"  Median: {np.median(text_lengths):.1f} characters")
    
    print(f"\n🖼️ Image Size Statistics:")
    widths = [s[0] for s in image_sizes]
    heights = [s[1] for s in image_sizes]
    print(f"  Width: {min(widths)} - {max(widths)} px (mean: {np.mean(widths):.0f})")
    print(f"  Height: {min(heights)} - {max(heights)} px (mean: {np.mean(heights):.0f})")
    
    # Visualize distributions
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
    
    ax1.hist(text_lengths, bins=30, edgecolor='black', alpha=0.7)
    ax1.set_xlabel('Text Length (characters)')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Distribution of Text Lengths')
    ax1.grid(alpha=0.3)
    
    ax2.scatter(widths, heights, alpha=0.3)
    ax2.set_xlabel('Image Width (px)')
    ax2.set_ylabel('Image Height (px)')
    ax2.set_title('Image Dimensions')
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

analyze_dataset(data_pairs)

---

## 4. Preprocessing Pipeline 🔧

Implement the preprocessing pipeline from our architecture.

In [ ]:
class ImagePreprocessor:
    """Preprocessing pipeline for handwriting images."""
    
    def __init__(self, target_size=(384, 384)):
        self.target_size = target_size
    
    def preprocess(self, image: Image.Image) -> Image.Image:
        """Apply full preprocessing pipeline."""
        # Convert to numpy
        img = np.array(image)
        
        # Convert to grayscale if needed
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        # Denoise
        img = cv2.fastNlMeansDenoising(img, None, h=10)
        
        # Enhance contrast
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        img = clahe.apply(img)
        
        # Convert back to PIL
        img = Image.fromarray(img)
        
        # Resize
        img = img.resize(self.target_size, Image.LANCZOS)
        
        # Convert to RGB (TrOCR expects RGB)
        img = img.convert('RGB')
        
        return img

# Test preprocessing
if len(data_pairs) > 0:
    preprocessor = ImagePreprocessor()
    
    # Test on one image
    test_img = Image.open(data_pairs[0]['image'])
    processed_img = preprocessor.preprocess(test_img)
    
    # Visualize
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    ax1.imshow(test_img, cmap='gray')
    ax1.set_title('Original')
    ax1.axis('off')
    
    ax2.imshow(processed_img)
    ax2.set_title('Preprocessed')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Preprocessing pipeline working!")

---

## 5. Dataset Preparation 📊

Create PyTorch dataset and dataloaders for training.

In [ ]:
class HandwritingDataset(Dataset):
    """PyTorch dataset for handwriting OCR."""
    
    def __init__(self, data_pairs, processor, preprocessor=None, max_length=128):
        self.data_pairs = data_pairs
        self.processor = processor
        self.preprocessor = preprocessor
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data_pairs)
    
    def __getitem__(self, idx):
        pair = self.data_pairs[idx]
        
        # Load image
        image = Image.open(pair['image']).convert('RGB')
        
        # Preprocess if available
        if self.preprocessor:
            image = self.preprocessor.preprocess(image)
        
        # Load text
        with open(pair['text_file'], 'r', encoding='utf-8') as f:
            text = f.read().strip()
        
        # Process with TrOCR processor
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        
        # Tokenize text
        labels = self.processor.tokenizer(
            text,
            padding="max_length",
            max_length=self.max_length,
            truncation=True,
            return_tensors="pt"
        ).input_ids
        
        # Replace padding token id with -100 (ignored by loss)
        labels[labels == self.processor.tokenizer.pad_token_id] = -100
        
        return {
            "pixel_values": pixel_values.squeeze(),
            "labels": labels.squeeze()
        }

print("✅ Dataset class defined")

In [ ]:
# Split dataset
if len(data_pairs) > 0:
    # Shuffle
    random.shuffle(data_pairs)
    
    # Split 80/10/10
    n_total = len(data_pairs)
    n_train = int(0.8 * n_total)
    n_val = int(0.1 * n_total)
    
    train_pairs = data_pairs[:n_train]
    val_pairs = data_pairs[n_train:n_train+n_val]
    test_pairs = data_pairs[n_train+n_val:]
    
    print(f"📊 Dataset splits:")
    print(f"  Train: {len(train_pairs)} samples")
    print(f"  Val:   {len(val_pairs)} samples")
    print(f"  Test:  {len(test_pairs)} samples")
else:
    print("⚠️ No data pairs found - cannot split dataset")

---

## 6. Model Training 🚀

Fine-tune TrOCR model on our dataset.

In [ ]:
# Load TrOCR model and processor
model_name = "microsoft/trocr-base-handwritten"

print(f"📥 Loading TrOCR model: {model_name}")
processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"✅ Model loaded on {device}")
print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")

In [ ]:
# Create datasets
if len(data_pairs) > 0:
    preprocessor = ImagePreprocessor()
    
    train_dataset = HandwritingDataset(train_pairs, processor, preprocessor)
    val_dataset = HandwritingDataset(val_pairs, processor, preprocessor)
    
    print(f"✅ Datasets created")
    print(f"  Train: {len(train_dataset)}")
    print(f"  Val: {len(val_dataset)}")
    
    # Test one batch
    sample = train_dataset[0]
    print(f"\n📊 Sample batch shape:")
    print(f"  Pixel values: {sample['pixel_values'].shape}")
    print(f"  Labels: {sample['labels'].shape}")

In [ ]:
# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./trocr-handwriting",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Adjust based on time/resources
    learning_rate=5e-5,
    warmup_steps=500,
    logging_steps=50,
    eval_steps=200,
    save_steps=200,
    evaluation_strategy="steps",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU available
    report_to="none",  # Disable wandb
    dataloader_num_workers=2,
)

print("✅ Training arguments configured")
print(f"\n🎯 Training configuration:")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Mixed precision: {training_args.fp16}")

In [ ]:
# Define compute metrics function
def compute_metrics(pred):
    """Compute CER and WER metrics."""
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    
    # Decode predictions and labels
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)
    
    # Calculate CER and WER
    cer_score = cer(label_str, pred_str)
    wer_score = wer(label_str, pred_str)
    
    return {
        "cer": cer_score,
        "wer": wer_score
    }

print("✅ Metrics function defined")

In [ ]:
# Create trainer
if len(data_pairs) > 0:
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=processor.feature_extractor,
        compute_metrics=compute_metrics,
    )
    
    print("✅ Trainer created")
    print("\n🚀 Ready to start training!")
    print("\nNote: Training may take a while depending on dataset size and hardware.")
    print("      On Colab with GPU, expect ~1-2 hours for 3 epochs on medium dataset.")

In [ ]:
# Start training
if len(data_pairs) > 0:
    print("🚀 Starting training...\n")
    
    # Train
    train_result = trainer.train()
    
    print("\n✅ Training completed!")
    print(f"\n📊 Training results:")
    print(f"  Final loss: {train_result.training_loss:.4f}")
    print(f"  Training time: {train_result.metrics['train_runtime']:.1f}s")
    
    # Save model
    trainer.save_model("./trocr-handwriting-final")
    processor.save_pretrained("./trocr-handwriting-final")
    print("\n💾 Model saved to ./trocr-handwriting-final")

---

## 7. Evaluation 📈

Evaluate the model on test set and visualize results.

In [ ]:
# Evaluate on test set
if len(data_pairs) > 0 and len(test_pairs) > 0:
    test_dataset = HandwritingDataset(test_pairs, processor, preprocessor)
    
    print("📊 Evaluating on test set...\n")
    eval_results = trainer.evaluate(test_dataset)
    
    print("\n✅ Evaluation completed!")
    print(f"\n📊 Test Results:")
    print(f"  Loss: {eval_results['eval_loss']:.4f}")
    print(f"  CER: {eval_results['eval_cer']:.2%}")
    print(f"  WER: {eval_results['eval_wer']:.2%}")

In [ ]:
# Inference function
def predict_text(image_path, model, processor, device):
    """Predict text from image."""
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    preprocessor = ImagePreprocessor()
    image = preprocessor.preprocess(image)
    
    # Process
    pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)
    
    # Generate
    with torch.no_grad():
        generated_ids = model.generate(pixel_values, max_length=128)
    
    # Decode
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return generated_text

print("✅ Inference function defined")

In [ ]:
# Visualize predictions
if len(test_pairs) > 0:
    n_samples = min(6, len(test_pairs))
    samples = random.sample(test_pairs, n_samples)
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, pair in enumerate(samples):
        # Load image
        img = Image.open(pair['image'])
        
        # Load ground truth
        with open(pair['text_file'], 'r', encoding='utf-8') as f:
            ground_truth = f.read().strip()
        
        # Predict
        prediction = predict_text(pair['image'], model, processor, device)
        
        # Calculate CER
        cer_score = cer([ground_truth], [prediction])
        
        # Display
        axes[idx].imshow(img, cmap='gray')
        axes[idx].axis('off')
        
        title = f"Ground Truth: {ground_truth[:30]}...\n"
        title += f"Prediction: {prediction[:30]}...\n"
        title += f"CER: {cer_score:.2%}"
        
        axes[idx].set_title(title, fontsize=8)
    
    plt.tight_layout()
    plt.suptitle('Model Predictions on Test Set', fontsize=14, y=1.02)
    plt.show()

---

## 8. Ensemble Implementation 🎯

Implement simple ensemble (for demo - can extend with Donut/LayoutLMv3)

In [ ]:
class SimpleEnsemble:
    """Simple ensemble combining multiple predictions."""
    
    def __init__(self, model, processor, device, n_runs=3):
        self.model = model
        self.processor = processor
        self.device = device
        self.n_runs = n_runs
    
    def predict_ensemble(self, image_path):
        """Predict with ensemble (multiple runs with different seeds)."""
        predictions = []
        
        for i in range(self.n_runs):
            # Load and preprocess
            image = Image.open(image_path).convert('RGB')
            preprocessor = ImagePreprocessor()
            image = preprocessor.preprocess(image)
            
            # Process
            pixel_values = self.processor(image, return_tensors="pt").pixel_values.to(self.device)
            
            # Generate with different sampling
            with torch.no_grad():
                generated_ids = self.model.generate(
                    pixel_values,
                    max_length=128,
                    num_beams=5,
                    do_sample=(i > 0),  # First run deterministic, others with sampling
                    temperature=0.7 if i > 0 else 1.0
                )
            
            # Decode
            text = self.processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
            predictions.append(text)
        
        # Simple voting: return most common prediction
        from collections import Counter
        counts = Counter(predictions)
        best_prediction = counts.most_common(1)[0][0]
        confidence = counts.most_common(1)[0][1] / self.n_runs
        
        return {
            'prediction': best_prediction,
            'confidence': confidence,
            'all_predictions': predictions
        }

# Create ensemble
ensemble = SimpleEnsemble(model, processor, device, n_runs=3)
print("✅ Ensemble created")

In [ ]:
# Test ensemble
if len(test_pairs) > 0:
    test_sample = test_pairs[0]
    
    print("🔄 Testing ensemble...\n")
    result = ensemble.predict_ensemble(test_sample['image'])
    
    # Load ground truth
    with open(test_sample['text_file'], 'r', encoding='utf-8') as f:
        ground_truth = f.read().strip()
    
    print(f"📝 Ground Truth: {ground_truth}")
    print(f"\n🎯 Ensemble Prediction: {result['prediction']}")
    print(f"📊 Confidence: {result['confidence']:.2%}")
    print(f"\n🔍 All Predictions:")
    for i, pred in enumerate(result['all_predictions'], 1):
        print(f"  Run {i}: {pred}")
    
    # Display image
    img = Image.open(test_sample['image'])
    plt.figure(figsize=(10, 3))
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.title(f"Ensemble: {result['prediction']} (Conf: {result['confidence']:.2%})")
    plt.show()

---

## 9. Interactive Demo 🎮

Try the model on custom images!

In [ ]:
# Upload and test your own image
if IN_COLAB:
    from google.colab import files
    
    print("📤 Upload a handwritten image to test:")
    uploaded = files.upload()
    
    if uploaded:
        # Get first uploaded file
        filename = list(uploaded.keys())[0]
        
        # Predict
        print(f"\n🔍 Processing {filename}...\n")
        result = ensemble.predict_ensemble(filename)
        
        print(f"🎯 Prediction: {result['prediction']}")
        print(f"📊 Confidence: {result['confidence']:.2%}")
        
        # Display
        img = Image.open(filename)
        plt.figure(figsize=(10, 4))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Prediction: {result['prediction']}")
        plt.show()
else:
    print("⚠️ File upload only available in Colab")

---

## 10. Export Model 💾

Save the trained model for deployment.

In [ ]:
# Save final model
output_dir = "./socar_trocr_final"
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

print(f"✅ Model saved to {output_dir}")
print("\n📦 Files saved:")
for file in Path(output_dir).glob('*'):
    print(f"  - {file.name}")

In [ ]:
# Download model (Colab only)
if IN_COLAB:
    print("📦 Creating zip file for download...")
    !zip -r socar_trocr_model.zip {output_dir}
    
    from google.colab import files
    files.download('socar_trocr_model.zip')
    print("✅ Model ready for download!")

---

## 11. Summary & Next Steps 📝

In [ ]:
print("""
🎉 CONGRATULATIONS!

You've successfully:
✅ Downloaded a real handwriting dataset
✅ Implemented preprocessing pipeline
✅ Fine-tuned TrOCR model
✅ Created an ensemble system
✅ Evaluated on test data
✅ Saved the trained model

📊 Final Results:
  • CER: {:.2%}
  • WER: {:.2%}
  • Model ready for deployment!

🚀 Next Steps for SOCAR Hackathon:

1. Fine-tune on SOCAR-specific data
   - Collect internal handwriting samples
   - Annotate with ground truth
   - Re-run this notebook

2. Add Donut model
   - For form field extraction
   - Follow similar training approach

3. Implement LayoutLMv3
   - For structured documents
   - Requires bounding boxes

4. Build production API
   - Flask/FastAPI endpoint
   - Docker container
   - Deploy to server

5. Create web demo
   - Gradio interface (from repo)
   - User-friendly UI

📚 Resources:
  • Full documentation: /docs folder
  • Model guide: /docs/model_design.md
  • Architecture: /docs/ARCHITECTURE.md

Good luck at the hackathon! 🏆
""".format(
    eval_results.get('eval_cer', 0.0) if len(test_pairs) > 0 else 0.0,
    eval_results.get('eval_wer', 0.0) if len(test_pairs) > 0 else 0.0
))